In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import altair as alt

from PIL import Image

In [2]:
df_ndcg = pd.read_csv('../resources/eval/ndcg.csv')

In [3]:
df_ndcg.head(3)

,Query ID,Ranking Method,User ID,NDCG@10,NDCG@5,NDCG@3
0,Q1,Average,U1,0.917250,0.805861,0.750000
1,Q1,Discounted Reward Only,U1,0.959709,0.860806,0.916667
2,Q1,Discounted Reward with Adjustment,U1,0.929303,0.797663,0.812500


In [4]:
#df_ndcg = df_ndcg[df_ndcg['User ID']!='U2']

In [5]:
#df_ndcg = df_ndcg[df_ndcg['Query ID']!='Q10']

In [6]:
df_ndcg_avg = df_ndcg.groupby(by=['Query ID', 'Ranking Method']).mean().reset_index()

In [7]:
df_ndcg_avg.head(10)

,Query ID,Ranking Method,NDCG@10,NDCG@5,NDCG@3
0,Q1,Average,0.927088,0.819196,0.794263
1,Q1,Discounted Reward Only,0.951410,0.852802,0.880234
2,Q1,Discounted Reward with Adjustment,0.943484,0.836534,0.836459
3,Q2,Average,0.949599,0.875985,0.851194
4,Q2,Discounted Reward Only,0.923657,0.805725,0.791626
5,Q2,Discounted Reward with Adjustment,0.917472,0.802590,0.771102
6,Q3,Average,0.945217,0.882151,0.857248
7,Q3,Discounted Reward Only,0.956361,0.897332,0.886685
8,Q3,Discounted Reward with Adjustment,0.951456,0.872912,0.871350
9,Q4,Average,0.907678,0.779336,0.749906


In [8]:
df_ndcg_avg.groupby(by='Ranking Method').mean()

,NDCG@10,NDCG@5,NDCG@3
Ranking Method,,,
Average,0.934942,0.847391,0.819158
Discounted Reward Only,0.938667,0.861454,0.836880
Discounted Reward with Adjustment,0.941133,0.861850,0.837094


In [9]:
df_ndcg_avg['Query ID'] = df_ndcg_avg['Query ID'].str[1:].astype(int)

In [10]:
df_ndcg_avg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Query ID        27 non-null     int32  
 1   Ranking Method  27 non-null     object 
 2   NDCG@10         27 non-null     float64
 3   NDCG@5          27 non-null     float64
 4   NDCG@3          27 non-null     float64
dtypes: float64(3), int32(1), object(1)
memory usage: 1.1+ KB


In [11]:
alt.Chart(data=df_ndcg_avg).mark_line().encode(
    x=alt.X('Query ID:O'),
    y=alt.Y('NDCG@10:Q').scale(domain=(0.7, 1.1)),
    color=alt.Color('Ranking Method:N')
).properties(
    width=600,
    height=300,
)

alt.Chart(...)

In [12]:
def plot_distances(df_input, title=''):
    df = df_input.copy()
    df = df[['distance_original', 'distance_opposite']]
    df['distance_original'] = df['distance_original'].astype('float64')
    df['distance_opposite'] = df['distance_opposite'].astype('float64')
    df = df.reset_index(drop=True).reset_index()
    df.columns=['index', 'original_query', 'opposite_query']
    df = df.melt(id_vars=['index'])

    domain = ['original_query', 'opposite_query']
    range_ = ['dodgerblue', 'coral']

    chart = alt.Chart(data=df).mark_line().encode(
            x=alt.X('index:Q').title('Record Index'),
            y=alt.Y("value:Q").scale(domain=(0,0.4)).title('Cosine Distance'),
            color=alt.Color("variable:N").scale(domain=domain, range=range_)
        ).properties(
            width=350,
            height=150,
            title=title
        )
    
    return chart